### RECSYS Домашка 01

In [251]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [252]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [253]:
# Добавляем поле "Год выхода на экраны"

In [256]:
movies['year'] = movies['title'].str.extract(r'(\d{4})')

In [257]:
movies['year'] = pd.to_numeric(movies['year'])

In [258]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II (1995),Comedy,1995.0


In [259]:
#Считаем средний рейтин и кол-во голосов по фильму

In [260]:
ratings1 = ratings.groupby('movieId').agg({'rating':'mean', 'userId':'count'})
ratings1 = ratings1.rename(columns = {'userId':'votes'})

In [261]:
ratings1.describe()

,rating,votes
count,9724.000000,9724.000000
mean,3.262448,10.369807
std,0.869874,22.401005
min,0.500000,1.000000
25%,2.800000,1.000000
50%,3.416667,3.000000
75%,3.911765,9.000000
max,5.000000,329.000000


In [262]:
tags1 = tags.groupby('movieId').agg({'tag':'count'})
tags1 = tags1.rename(columns = {'tag':'tags'})

In [263]:
tags1.head()

,tags
movieId,
1,3
2,4
3,2
5,2
7,1


In [264]:
mintags = tags1['tags'].min()
mintags

1

In [265]:
# Объединяем фильмы и рейтинг

In [266]:
ratings2 = pd.merge(ratings1, movies, on='movieId', how='inner')

In [267]:
# Добавляем тэги

In [268]:
ratings2 = pd.merge(ratings2, tags1, on='movieId', how='left')
ratings2 = ratings2.fillna(mintags)

In [269]:
ratings2.head()

,movieId,rating,votes,title,genres,year,tags
0,1,3.920930,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,3.0
1,2,3.431818,110,Jumanji (1995),Adventure|Children|Fantasy,1995.0,4.0
2,3,3.259615,52,Grumpier Old Men (1995),Comedy|Romance,1995.0,2.0
3,4,2.357143,7,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,1.0
4,5,3.071429,49,Father of the Bride Part II (1995),Comedy,1995.0,2.0


In [270]:
# Нормализуем голоса и кол-во тэгов, 
# делаем метрику голосов в диапазоне 0.3-1.0,
# метрику тэгов  в диапазоне 0.7-1.0,
# Это определяет как каждая из метрик влияет на распреление фильмов в топе

In [271]:
votesPart = 0.7
tagsPart = 0.3

In [272]:
ratings2['votes_norm'] = (1.0 - votesPart) + (ratings2['votes'] - ratings2['votes'].min())/(ratings2['votes'].max() - ratings2['votes'].min()) * votesPart
ratings2['tags_norm'] = (1.0 - tagsPart) + (ratings2['tags'] - ratings2['tags'].min())/(ratings2['tags'].max() - ratings2['tags'].min()) * tagsPart

In [273]:
ratings2.head()

,movieId,rating,votes,title,genres,year,tags,votes_norm,tags_norm
0,1,3.920930,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,3.0,0.756707,0.703333
1,2,3.431818,110,Jumanji (1995),Adventure|Children|Fantasy,1995.0,4.0,0.532622,0.705000
2,3,3.259615,52,Grumpier Old Men (1995),Comedy|Romance,1995.0,2.0,0.408841,0.701667
3,4,2.357143,7,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,1.0,0.312805,0.700000
4,5,3.071429,49,Father of the Bride Part II (1995),Comedy,1995.0,2.0,0.402439,0.701667


In [274]:
# Вычисляем общую метрику "Рейтинг * Голоса * Тэги"

In [275]:
ratings2['wa'] = ratings2['rating'] * ratings2['votes_norm'] * ratings2['tags_norm']

In [276]:
ratings2.head()

,movieId,rating,votes,title,genres,year,tags,votes_norm,tags_norm,wa
0,1,3.920930,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,3.0,0.756707,0.703333,2.086788
1,2,3.431818,110,Jumanji (1995),Adventure|Children|Fantasy,1995.0,4.0,0.532622,0.705000,1.288642
2,3,3.259615,52,Grumpier Old Men (1995),Comedy|Romance,1995.0,2.0,0.408841,0.701667,0.935087
3,4,2.357143,7,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,1.0,0.312805,0.700000,0.516128
4,5,3.071429,49,Father of the Bride Part II (1995),Comedy,1995.0,2.0,0.402439,0.701667,0.867304


In [277]:
# Топ 10

In [278]:
ratings2.sort_values(by=['wa'], ascending=[0]).head(10)

,movieId,rating,votes,title,genres,year,tags,votes_norm,tags_norm,wa
257,296,4.197068,307,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994.0,181.0,0.953049,1.000000,4.000011
277,318,4.429022,317,"Shawshank Redemption, The (1994)",Crime|Drama,1994.0,4.0,0.974390,0.705000,3.042495
314,356,4.164134,329,Forrest Gump (1994),Comedy|Drama|Romance|War,1994.0,9.0,1.000000,0.713333,2.970415
1938,2571,4.192446,278,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999.0,5.0,0.891159,0.706667,2.640201
510,593,4.161290,279,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,1991.0,6.0,0.893293,0.708333,2.633052
224,260,4.231076,251,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1977.0,26.0,0.833537,0.741667,2.615678
2224,2959,4.272936,218,Fight Club (1999),Action|Crime|Drama|Thriller,1999.0,54.0,0.763110,0.788333,2.570533
97,110,4.031646,237,Braveheart (1995),Action|Drama|War,1995.0,10.0,0.803659,0.715000,2.316647
461,527,4.225000,220,Schindler's List (1993),Drama|War,1993.0,7.0,0.767378,0.710000,2.301942
897,1196,4.215640,211,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1980.0,10.0,0.748171,0.715000,2.255123


In [279]:
# Топ 10 для фильмов "action + comedy"

In [280]:
ratings2[ratings2['genres'].str.contains(r'(?i)(action|comedy)')].sort_values(by=['wa'], ascending=[0]).head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,movieId,rating,votes,title,genres,year,tags,votes_norm,tags_norm,wa
257,296,4.197068,307,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994.0,181.0,0.953049,1.000000,4.000011
314,356,4.164134,329,Forrest Gump (1994),Comedy|Drama|Romance|War,1994.0,9.0,1.000000,0.713333,2.970415
1938,2571,4.192446,278,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999.0,5.0,0.891159,0.706667,2.640201
224,260,4.231076,251,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1977.0,26.0,0.833537,0.741667,2.615678
2224,2959,4.272936,218,Fight Club (1999),Action|Crime|Drama|Thriller,1999.0,54.0,0.763110,0.788333,2.570533
97,110,4.031646,237,Braveheart (1995),Action|Drama|War,1995.0,10.0,0.803659,0.715000,2.316647
897,1196,4.215640,211,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1980.0,10.0,0.748171,0.715000,2.255123
507,589,3.970982,224,Terminator 2: Judgment Day (1991),Action|Sci-Fi,1991.0,8.0,0.775915,0.711667,2.192747
899,1198,4.207500,200,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1981.0,8.0,0.724695,0.711667,2.169982
418,480,3.750000,238,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,1993.0,1.0,0.805793,0.700000,2.115206


In [281]:
# Топ 10 для фильмов XXI Века

In [282]:
ratings2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 0 to 9723
Data columns (total 10 columns):
movieId       9724 non-null int64
rating        9724 non-null float64
votes         9724 non-null int64
title         9724 non-null object
genres        9724 non-null object
year          9724 non-null float64
tags          9724 non-null float64
votes_norm    9724 non-null float64
tags_norm     9724 non-null float64
wa            9724 non-null float64
dtypes: float64(6), int64(2), object(2)
memory usage: 835.7+ KB


In [284]:
ratings2[ratings2['year'] >= 2000].sort_values(by=['wa'], ascending=[0]).head(10)

,movieId,rating,votes,title,genres,year,tags,votes_norm,tags_norm,wa
3633,4993,4.106061,198,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,2001.0,7.0,0.720427,0.710000,2.100263
4791,7153,4.118919,185,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,2003.0,10.0,0.692683,0.715000,2.039970
4131,5952,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,2002.0,2.0,0.699085,0.701667,1.972536
3136,4226,4.122642,159,Memento (2000),Mystery|Thriller,2000.0,13.0,0.637195,0.720000,1.891387
2670,3578,3.938235,170,Gladiator (2000),Action|Adventure|Drama,2000.0,8.0,0.660671,0.711667,1.851669
6693,58559,4.238255,149,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,2008.0,4.0,0.615854,0.705000,1.840152
7355,79132,4.066434,143,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,2010.0,26.0,0.603049,0.741667,1.818758
4900,7361,4.160305,131,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi,2004.0,34.0,0.577439,0.755000,1.813754
3189,4306,3.867647,170,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...,2001.0,1.0,0.660671,0.700000,1.788669
4354,6377,3.960993,141,Finding Nemo (2003),Adventure|Animation|Children|Comedy,2003.0,3.0,0.598780,0.703333,1.668142
